<a href="https://colab.research.google.com/github/Yujini68/ESAA/blob/main/%EA%B3%BC%EC%A0%9C_0913.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 7 앙상블 학습과 랜덤 포레스트

## 7.1 투표 기반 분류기

직접 투표 분류기 : 다수결 투표로 정해지는 분류기

약한 학습기 : 랜덤 추측보다 조금 더 높은 성능을 내는 분류기
강한 학습기 : 높은 정확도를 내는 분류기

큰 수의 법칙으로, 약한 학습기가 모이면 앙상블은 강한 학습기가 될 수 있음

- 여러 분류기를 조합하여 사이킷런의 투표 기반 분류기를 만들고 훈련

In [19]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os

# 5장에서 소개한 moons 데이터셋 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

각 분류기의 테스트셋 정확도 확인

In [21]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.95
SVC 0.95
VotingClassifier 0.95


투표 기반 분류기가 다른 개별 분류기보다 성능이 조금 더 높음

## 7.2 배깅과 페이스팅

배깅 : 훈련 세트에서 중복을 허용하여 샘플링하는 방식
페이스팅 : 중복을 허용하지 않고 샘플링하는 방식

### 7.2.1 사이킷런의 배깅과 페이스팅

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)   #배깅 : bootstrap = True / 페이스팅 : False
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

### 7.2.2 oob 평가

각 예측기에 선택되지 않은 나머지 훈련 샘플을 oob 샘플이라고 하며 이를 사용해 평가함

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    bootstrap = True, n_jobs = -1, oob_score = True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9

In [6]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

In [7]:
#oob 샘플에 대한 결정 함수의 값
bag_clf.oob_decision_function_

array([[0.53608247, 0.46391753],
       [1.        , 0.        ],
       [0.01111111, 0.98888889],
       [0.07486631, 0.92513369],
       [0.62686567, 0.37313433],
       [0.50753769, 0.49246231],
       [0.75428571, 0.24571429],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.0104712 , 0.9895288 ],
       [0.98918919, 0.01081081],
       [0.0802139 , 0.9197861 ],
       [0.88372093, 0.11627907],
       [0.08383234, 0.91616766],
       [0.51075269, 0.48924731],
       [0.87027027, 0.12972973],
       [0.        , 1.        ],
       [0.02538071, 0.97461929],
       [0.94972067, 0.05027933],
       [0.01724138, 0.98275862],
       [0.98870056, 0.01129944],
       [0.97487437, 0.02512563],
       [1.        , 0.        ],
       [0.53333333, 0.46666667],
       [0.73743017, 0.26256983],
       [0.01754386, 0.98245614],
       [0.25966851, 0.74033149],
       [0.98913043, 0.01086957],
       [1.        , 0.        ],
       [0.96825397, 0.03174603],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링하는 것
랜덤 서브스페이스 방식 : 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것
- 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춤

## 7.4 랜덤 포레스트

배깅 방법 (또는 페이스팅)을 적용한 결정 트리의 앙상블

500개 트리로 이뤄진 랜덤 포레스트 분류기를 여러 CPU 코어에서 훈련

In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes=16, n_jobs = -1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features = "auto", max_leaf_nodes = 16),
    n_estimators = 500, max_samples = 1.0, bootstrap = True, n_jobs = -1)

### 7.4.1 엑스트라 트리

익스트림 랜덤 트리 앙상블, 엑스트라 트리 : 극단적으로 무작위한 트리의 랜덤 포레스트

- 편향이 늘어나지만 대신 분산을 낮추게 됨
- 일반적인 랜덤 포레스트보다 빠름

### 7.4.2 특성 중요도

- 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도 측정, 가중치 평균, 각 노드의 가중치는 연관된 훈련 샘플 수 와 같음
- 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화함

In [10]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)
rnd_clf.fit(iris['data'],iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name,score)

sepal length (cm) 0.10526362544541756
sepal width (cm) 0.025297460055350576
petal length (cm) 0.42629974064008974
petal width (cm) 0.44313917385914214


## 7.5 부스팅

약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법

### 7.5.1 에이다부스트

이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높여서 새로운 예측기를 학습하기 어려운 샘플에 점점 맞춤


In [11]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1), n_estimators = 200,
    algorithm = "SAMME.R", learning_rate = 0.5
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레이디언트 부스팅

앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가, 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킴

In [12]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)
y2 = y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [13]:
y3 = y2-tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [23]:
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators=2, learning_rate = 1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=2)

120개의 트리로 GBRT 앙상블을 훈련시키고 최적의 트리 수를 사용해 새로운 GBRT 앙상블을 훈련

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=119)

연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춤

In [17]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up +=1
    if error_going_up == 5:
      break #조기 종료

GradientBoostingRegressor : 각 트리가 훈련할 때 사용할 훈련 샘플의 비율 지정할 수 있는 subsample 매개변수도 지원, 편향이 높아지는 대신 분산이 낮아짐, 훈련 속도를 상당히 높임 -> 확률적 그레이디언트 부스

In [18]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred = xgb_reg.predict(X_val)

## 7.6 스태킹

앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련?

사이킷런에서는 직접 지원 X